# Objectives

## Pairwise Covariance

We want the covariance ($cov$) of diseases $x$ and $y$ for all diseases in the database. Furthermore, these covariances are split by sex $s$ and age ranges $l$ and $h$.

Diseases $x$ and $y$ are specified using ICD codes. Sexes $s$ are either $female$ or $male$, and $l, h$ specify lower and upper bound ranges (ranging between 0 and 100). Hence a query is of the form $cov(x, y, s, l, h)$.

For example, a query for the covariance of disease with ICD code 100 and ICD code 200 for females aged between 10 and 25 would be:

$$cov(100, 200, female, 10, 25) = 0.25$$

indicating a covariance of 0.25.

## Covariance Matrices

We also want covariance matrices for all possible diseases for each of the two sexes for defined age ranges. In other words, a set of matrices $\mathbf{M}_{s, l, h}$ should be produced for the following age increments: 1, 5, 10. These should be cached for fast querying and exported in a machine readable format.

## Design Outcome

There should be two deliverables:

- A REST interface allowing for queries for **pairwise covariance** and the predetermined **covariance matrices**
- Machine readable output of **covariance matrices**.


# Data

We are given the following data sources (in the folder `/original_data/`:

- `adm3.sas7bdat` (53.5 MB): ignored for the purposes of this project
- `cancer3.sas7bdat` (1.2 MB): ignored for the purposes of this project
- `opd3.sas7bdat` (5.14 GB): data source for this project

## Format

The files are in SAS data format (ugh) which can be read in Python easily using the `sas7bdat` package.

## Headers

Using `sas7bdat` in command line, we can extract the headers of the `opd3.sas7bdat`.

In [12]:
from sas7bdat import SAS7BDAT as sas

with sas('original_data/opd3.sas7bdat') as f:
    print(f.header)

Header:
	col_count_p1: 24
	col_count_p2: 0
	column_count: 24
	compression: None
	creator: None
	creator_proc: None
	date_created: 2016-03-23 13:59:06.127000
	date_modified: 2016-03-23 14:43:56.712000
	endianess: little
	file_type: DATA
	filename: opd3.sas7bdat
	header_length: 65536
	lcp: 8
	lcs: 0
	mix_page_row_count: 322
	name: OPD3
	os_name: 
	os_type: 
	page_count: 78393
	page_length: 65536
	platform: windows
	row_count: 26653278
	row_length: 192
	sas_release: 9.0401M1
	server_type: X64_7PRO
	u64: False

Contents of dataset "OPD3":
Num Name        Type   Length Format Label
--- ----------- ------ ------ ------ -----
  1 FEEYR       number      8 BEST        
  2 ATYPE       number      8 BEST        
  3 APP_DAY     number      8 BEST        
  4 CTYPE       number      8 BEST        
  5 SN          number      8 BEST        
  6 CITY        number      8 BEST        
  7 HTYPE       number      8 BEST        
  8 HID         number      8 BEST        
  9 DID         string     10

The interpretation of these variables are:

- APP_DAY: Apply Date
- APP_DAY2: Apply Date (in SAS)
- ATYPE: Apply Type
- BIR: Birthday YM
- CTYPE: Case Type
- CITY:  The City Code of Hospital
- FEEYR: Fee Year/Month
- VSDAY: Visit Date
- VSTYPE:  Visit Department
- HTYPE: Level of Hospital
- HID: Code of Hospital
- OPC1: Code of Operation
- ICD1: Code of Diagnosis 1
- ICD2: Code of Diagnosis 2
- ICD3: Code of Diagnosis 3
- SEX: Gender
- PID: ID
- PDOT:  Copayment Dot
- PNO: Copayment Code
- DID: ID of Doctor
- SN:  Serial Number
- TADOT: Total Apply Dot
- TDOT: Total Dot
- YR: Year

## Preview

We can also preview the first few rows of the data.

In [16]:
import itertools
import pandas

pandas.set_option('display.max_columns', 999)

with sas('original_data/opd3.sas7bdat') as f:
    gen = f.readlines()
    headers = next(gen)
    rows = list(itertools.islice(gen, 0, 20))
    dataframe = pandas.DataFrame(rows, columns=headers)calc
dataframe

,FEEYR,ATYPE,APP_DAY,CTYPE,SN,CITY,HTYPE,HID,DID,VSTYPE,VSDAY,BIR,PNO,ICD1,ICD2,ICD3,OPC1,T_DOT,PDOT,TDOT,SEX,PID,APP_DAY2,YR
0,200602.0,1.0,20060309.0,19.0,297.0,3201.0,37.0,161403.0,wxtqaajejb,40.0,20060211.0,198207.0,H10,5243.0,NaN,None,,770.0,50.0,720.0,2.0,erlra,16869.0,2006.0
1,200605.0,1.0,20060607.0,1.0,964.0,2101.0,35.0,55584.0,ccopskyrqu,11.0,20060510.0,198207.0,D10,7080.0,NaN,None,,396.0,50.0,346.0,2.0,erlra,16959.0,2006.0
2,200607.0,1.0,20060807.0,1.0,215.0,203.0,25.0,338027.0,gxurzkwtcm,0.0,20060720.0,197108.0,D10,460.0,NaN,None,,425.0,50.0,375.0,1.0,erlsa,17020.0,2006.0
3,200603.0,1.0,20060403.0,1.0,1080.0,1705.0,35.0,71180.0,vatnpkveaw,0.0,20060318.0,199107.0,D10,462.0,NaN,None,,396.0,50.0,346.0,2.0,eihya,16894.0,2006.0
4,200603.0,1.0,20060403.0,1.0,1202.0,1705.0,35.0,71180.0,ivdinwvrlk,0.0,20060320.0,199107.0,D10,4659.0,NaN,None,,396.0,50.0,346.0,2.0,eihya,16894.0,2006.0
5,200607.0,1.0,20060814.0,9.0,537.0,1705.0,35.0,38728.0,ahnakixwyl,9.0,20060730.0,199107.0,D10,4619.0,NaN,None,,465.0,70.0,395.0,2.0,eihya,17027.0,2006.0
6,200607.0,1.0,20060814.0,9.0,303.0,1705.0,35.0,38728.0,ahnakixwyl,9.0,20060718.0,199107.0,D10,4619.0,38102.0,None,,539.0,70.0,469.0,2.0,eihya,17027.0,2006.0
7,200607.0,1.0,20060814.0,9.0,419.0,1705.0,35.0,38728.0,ahnakixwyl,9.0,20060724.0,199107.0,D10,38102.0,4619.0,None,,1261.0,70.0,1191.0,2.0,eihya,17027.0,2006.0
8,200607.0,1.0,20060814.0,9.0,236.0,1705.0,35.0,38728.0,ahnakixwyl,9.0,20060714.0,199107.0,D10,4619.0,3829.0,None,,538.0,70.0,468.0,2.0,eihya,17027.0,2006.0
9,200607.0,1.0,20060814.0,9.0,505.0,1705.0,35.0,38728.0,ahnakixwyl,9.0,20060727.0,199107.0,D10,4619.0,NaN,None,,531.0,70.0,461.0,2.0,eihya,17027.0,2006.0


# Covariance Calculation

## Binary Distribution Sample Covariance

Courtesy of Eric S. Wu, we can calculate covariance using the binary distribution sample covariance formula:

$$cov(a, b) = \frac{nk_{a,b} - k_a k_b}{n(1-n)}$$

where $k_{a,b}$ is the number of cases where $a$ and $b$ co-occurred, $k_{a}$ is the number of cases where $a$ occurred and $k_{b}$ is the number of cases where $b$ occurred. $n$ is the total number of cases.

In our case, since covariance has three more parameters $s$, $l$, and $h$, we produce the formula:

$$cov(x, y, s, l, h) = \frac{n_{s, l, h}k_{x, y, s, l, h} - k_{x, s, l, h} k_{y, s, l, h}}{n_{s, l, h}(1-n_{s, l, h})}$$

This simply further parameterizes the variables across the 3 dimensions $s$, $l$, and $h$.

# System Design

## Dimensionality

There are $\frac{100}{2}(100) = 5,000$ combinations of $l, h$ (since $l \in [0, 100]$ and $h \in [l, 100]$.) There are two combinations of gender ($female$ or $male$). Hence, there are 10,000 different $s, l, h$ combinations. There are 9,787 different ICD codes in the database. This means that the dimensionality of the $(x, s, l, h)$ tuple is around $10,000 * 10,000 = 100,000,000$ and the dimensionality of the $(x, y, s, l, h)$ tuple is $10,000 * 10,000 * 10,000$. It is then intractible to store every $k_{x, y, s, l, h}$ and $k_{x, s, l, h}$ in a database.

## Design Solution

However, a database with hashed indices on ICD code, sex, and age can be used easily to calculate pairwise covariance. To produce the covariance matrices, a separate in-memory database can be used to store the raw data and pairwise covariance queries can be made. Alternatively, this problem is trivially parallelizable (since we are making separate independent queries) in-memory. We can consider a CUDA implementation given time.

## Database Schema

This means that the only relevant data we need to retain in our database is:

- Patient ID. This is given in the original database as `PID`
- ICD Code. However, ICD Code is currently given as `ICD1`, `ICD2`, and `ICD3`. This represents the case of a single individual being given multiple valid diagnoses in a single visit. This complicates our database unnecessarily, and we coerce cases with multiple diagnoses into multiple cases (rows). This results in each row having only `ICD`.
- Sex, which is given in the original data as `SEX` (duh)
- Age, which need to be calculated for each row of data. This can be calculated from the visit date of the patient (`VSDAY`) and the birth date of the patient (`BIR`). However, since `VSDAY` is given as `YYYYMMDD` and `BIR` is given as `YYYYMM`, we simply take `AGE = VSDAY / 10000 - BIR / 100`. This produces `AGE` in integers.

Then, we create a single B-tree on the tuple `(ICD, SEX, AGE)`. We now have the schema:

```SQL
CREATE SEQUENCE patient_id_seq;

CREATE TABLE patients (
  ID  INTEGER DEFAULT nextval('patient_id_seq'),
  PID VARCHAR(11) NOT NULL,
  ICD INTEGER NOT NULL,
  SEX INTEGER NOT NULL,
  AGE INTEGER NOT NULL,
  PRIMARY KEY (ID)
)

CREATE INDEX ICD_SEX_AGE ON patients (ICD, SEX, AGE)
```

This allows us to query quickly the number of patients with a specific disease for a specific sex for a specific age using

```SQL
WITH pid_x AS (
  SELECT DISTINCT PID FROM patients WHERE ICD=100 AND SEX=1 AND AGE>=10 AND AGE<20 GROUP BY PID
), pid_y AS (
  SELECT DISTINCT PID FROM patients WHERE ICD=200 AND SEX=1 AND AGE>=10 AND AGE<20 GROUP BY PID
), pid_xy AS (
  SELECT DISTINCT pid_x.PID FROM pid_x, pid_y WHERE pid_x.PID=pid_y.PID GROUP BY pid_x.PID
)
SELECT COUNT(pid_x.PID) as kx, COUNT(pid_y.PID) as ky, COUNT(pid_xy.PID) as kxy
FROM pid_x, pid_y, pid_xy
```

Furthermore, we cache the $n_{s, l, h}$ values:

```SQL
CREATE TABLE patient_counts(
  AGE INTEGER NOT NULL,
  SEX INTEGER NOT NULL,
  N INTEGER NOT NULL,
  PRIMARY KEY (AGE)
)
```

and these values can be extracted quickly via:

```SQL
SELECT SUM(N)
FROM patient_counts
WHERE SEX=sex AND AGE>=l AND AGE<h;
```

# Database Setup

## Conversion to CSV

The data given is in `.sas7bdat` which is painful to work with. Converting the entirety to `.csv` using `sas7bdat_to_csv` provided in the `sas7bdat` python package converted the 5 GB SAS data to 4 GB of CSV.

## Create CSVs to Populate DB

This section is done in `create_db_data.py` in the root directory.

We want to iterate through each row of data, doing the following:

1. Retain only the columns `[PID, ICD1, ICD2, ICD3, SEX, VSDAY, BIR]`
2. Calculate `AGE = AGE = VSDAY / 10000 - BIR / 100` since `VSDAY` is given as `YYYYMMDD` and `BIR` is given as `YYYYMM`, we simply take `AGE = VSDAY / 10000 - BIR / 100`. This produces `AGE` in integers.
3. If `ICD2` or `ICD3` is populated, discard them and create additional rows using the `ICD2` and/or `ICD3` values as `ICD1` values for the new rows.
4. Rename `ICD1` to `ICD`.

This logic is in `parse/parse_raw.py`.

This produces `csv_data/patients.csv` (682.5 MB).

We also want to calculate patient counts to populate the `patient_counts` table. To do this, we will count the number of patients with a particular age.

This produces `csv_data/patient_counts.csv`.

## Creating a DB Instance

I launched a Amazon RDS instance. Read-only credentials to the instance are available:

- `host`: `pwas-postgres.c5fx9eaq2kdo.us-east-1.rds.amazonaws.com`
- `port`: 5432
- `login`: `guest`
- `password`: `guest`

For admin privileges, contact the author.

Tables and sequences created are in `create_table.sql`.